# Create a Q&A Chatbot with LangChain Project

### Set the OpenAI API Key as an Environment Variable

In [ ]:
%load_ext dotenv
%dotenv

### Import the Libraries

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader

from langchain_text_splitters import (MarkdownHeaderTextSplitter, 
                                      TokenTextSplitter)

from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate,
                                    HumanMessagePromptTemplate, 
                                    ChatPromptTemplate)
from langchain_core.runnables import (RunnablePassthrough, 
                                      RunnableLambda, 
                                      chain)

from langchain_openai import (ChatOpenAI, 
                              OpenAIEmbeddings)

from langchain_chroma.vectorstores import Chroma

### Load the Course Transcript

In [ ]:
loader_pdf = 

In [ ]:
docs_list = 

In [ ]:
string_list_concat = 

### Split the Course Transcript with MarkdownHeaderTextSplitter

In [ ]:
md_splitter = 

In [ ]:
docs_list_md_split = 

### Create a Chain to Correct the Course Transcript

In [ ]:
string_list_split = 

In [ ]:
PROMPT_FORMATTING_S = '''Improve the following Tableau lecture transcript by:
- Splitting the text into meaningful paragraphs
- Correcting any misplaced punctuation
- Fixing mistranscribed words (e.g., changing 'tableaux' to 'Tableau')"
'''

PROMPT_TEMPLATE_FORMATTING_H = '''This is the transcript:
{lecture_transcript}
'''

In [ ]:
prompt_formatting_s = 
prompt_template_formatting_h = 
chat_prompt_template_formatting = 

In [ ]:
chat = 

In [ ]:
str_output_parser = 

In [ ]:
chain_formatting = 

In [ ]:
string_list_formatted = 

In [ ]:
# Override the docs_list_md_split list such that the page_content parameter of each Document objects stores the updated lecture.


### Split the Lectures with TokenTextSplitter

In [ ]:
token_splitter = 

In [ ]:
docs_list_tokens_split = 

### Create Embeddings, Vector Store, and Retriever

In [ ]:
embedding = 

In [ ]:
vectorstore = 

In [ ]:
retriever = 

### Create Prompts and Prompt Templates for the Q&A Chatbot Chain

In [ ]:
PROMPT_CREATING_QUESTION = '''Lecture: {question_lecture}
Title: {question_title}
Body: {question_body}'''

PROMPT_RETRIEVING_S = '''You will receive a question from a student taking a Tableau course, which includes a title and a body. 
The corresponding lecture will also be provided.

Answer the question using only the provided context.

At the end of your response, include the section and lecture names where the context was drawn from, formatted as follows: 
Resources: 
Section: *Section Title*, Lecture: *Lecture Title* 
...
Replace *Section Title* and *Lecture Title* with the appropriate titles.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

prompt_creating_question = 
prompt_retrieving_s = 
prompt_template_retrieving_h = 

chat_prompt_template_retrieving = 

### Create the First Version of the Q&A Chatbot Chain

In [ ]:
chain_retrieving = 

In [ ]:
result = chain_retrieving.invoke({"question_lecture": "Adding a custom calculation",
                                  "question_title": "Why are we using SUM here? It's unclear to me.",
                                  "question_body": "This question refers to calculating the GM%."})

In [ ]:
result

### Create a Runnable Function to Format the Context

In [ ]:
def format_context(dictionary):
    pass

In [ ]:
chain_retrieving_improved = 

In [ ]:
result_improved = chain_retrieving_improved.invoke({"question_lecture": "Adding a custom calculation",
                                                    "question_title": "Why are we using SUM here? It's unclear to me.",
                                                    "question_body": "This question refers to calculating the GM%."})

In [ ]:
result_improved

### Stream the Response

In [ ]:
result_streamed = 

In [ ]:
# Create a for-loop to stream the response
